[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/experimental/algos-and-libraries/bertopic/06_bertopic_params.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/experimental/algos-and-libraries/bertopic/06_bertopic_params.ipynb)

# BERTopic Parameters

In this final notebook, we take what we have learned in the first five notebooks and apply this directly within the BERTopic library.

In [1]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/reddit-topics', split='train',
    revision='c14d532'  # this specifies current version of dataset
)
# remove short self text
data = data.filter(lambda x: True if len(x['selftext']) > 30 else 0)
data

Using custom data configuration jamescalam--reddit-topics-36b2068b7cd7acf8
Reusing dataset json (/Users/jamesbriggs/.cache/huggingface/datasets/json/jamescalam--reddit-topics-36b2068b7cd7acf8/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
Loading cached processed dataset at /Users/jamesbriggs/.cache/huggingface/datasets/json/jamescalam--reddit-topics-36b2068b7cd7acf8/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-50063e96c8952ccf.arrow


Dataset({
    features: ['sub', 'title', 'selftext', 'upvote_ratio', 'id', 'created_utc'],
    num_rows: 3118
})

To pass the level of custom parameters the we used in the previous notebooks to BERTopic, we actually initialize our own instances of UMAP and HDBSCAN.

In [2]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        prediction_data=True, gen_min_span_tree=True)

***NOTE**: We need to set `prediction_data=True` otherwise we return **AttributeError: No prediction data was generated***

Then we pass these to the `umap_model` and `hdbscan_model` arguments when initializing `BERTopic`. In the previous notebook we used the `sentence-transformers/all-MiniLM-L6-v2` embedding model, we do the same here by passing this string to the `embedding_model` argument. Finally, we add in the `CountVectorizer` which removes English stopwords for us in the c-TF-IDF step.

In [3]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com']

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
#model.fit(data['selftext'])
topics, probs = model.fit_transform(data['selftext'])

Batches: 100%|██████████| 98/98 [06:44<00:00,  4.13s/it]
2022-05-03 17:55:08,014 - BERTopic - Transformed documents to Embeddings
OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-05-03 17:55:16,971 - BERTopic - Reduced dimensionality
2022-05-03 17:55:17,107 - BERTopic - Clustered reduced embeddings


In [4]:
for i in range(5):
    print(f"{topics[i]}: {data['title'][i]}")

0: Followup- Here are the results so far from my algo's price calculations on the symbols you gave me:
0: This 2 hr Mohnish Pabrai lecture is better than 90% of my Ivy League Investing Classes
0: Vital Energy $VUX has “more than doubled production” but the market hasn’t noticed this yet – it was not yet in financials. Easy cashflow, more growth potential and a future stock multibagger using Q4 2021 numbers financials and 900 BOE/D production + higher oil prices of $100 WTI today
0: Keep averaging down or wait?
0: The future of credit card companies vs. ApplePay, Cash App, Venmo, etc.


In [5]:
model.visualize_barchart()

In [6]:
model.visualize_hierarchy()